### L9 Cert Work
importing libraries
pandas for data frames and matplotlib for visualizations and graphs

In [1]:
import pandas as pd
import matplotlib as plt

loading and inspecting the dataset (first 10rows)

In [2]:
tm=pd.read_csv("TMQ10_overseas_trips.csv")
tm.head(10)

,Statistic Label,Area of Residence,Quarter,UNIT,VALUE
0,Overseas Trips to Ireland,All trips to Ireland,2008Q1,Thousand,1542.2
1,Overseas Trips to Ireland,All trips to Ireland,2008Q2,Thousand,2159.4
2,Overseas Trips to Ireland,All trips to Ireland,2008Q3,Thousand,2472.4
3,Overseas Trips to Ireland,All trips to Ireland,2008Q4,Thousand,1665.0
4,Overseas Trips to Ireland,All trips to Ireland,2009Q1,Thousand,1402.2
5,Overseas Trips to Ireland,All trips to Ireland,2009Q2,Thousand,1901.9
6,Overseas Trips to Ireland,All trips to Ireland,2009Q3,Thousand,2177.3
7,Overseas Trips to Ireland,All trips to Ireland,2009Q4,Thousand,1446.1
8,Overseas Trips to Ireland,All trips to Ireland,2010Q1,Thousand,1084.8
9,Overseas Trips to Ireland,All trips to Ireland,2010Q2,Thousand,1541.4
